#### Student Name:
#### Student ID:

# Assignment 4

### Markov Chain, LZify

Instructions: 

* This notebook is an interactive assignment; please read and follow the instructions in each cell. 

* Cells that require your input (in the form of code or written response) will have 'Question #' above.

* After completing the assignment, please submit this notebook as a PDF and a WAV file of your Fur Elise variation.

* Make sure to mark the page with your solution for each problem on Gradescope. Any problems without the correct pages marked may receive a score of 0. 

## Markov-Based Chord Prediction

In music, certain chord transitions are more likely than others. This idea can be applied as Markov Models, where the first-order temporal relationships between the various chords are captured by the transition probability matrix $A$. 

If we consider only major and minor chords, there are a total of 24 chords in this model (12 major, from C through B, and 12 minor, from C through B), formalized as:

\begin{equation}
\label{eq:ChordReco:HMM:App:Spec:SetStates}
  \mathcal{A} = \{\mathbf{C},\mathbf{C}^\sharp,\ldots,\mathbf{B},\mathbf{Cm},\mathbf{Cm^\sharp},\ldots,\mathbf{Bm}\} 
\end{equation}

We use the notation $\alpha{i}\rightarrow\alpha{j}$ referring to the transition from state $\alpha{i}$ to state $\alpha{j}$, $i,j\in[1:24]$. For example, the coefficient $a_{1,2}$ expresses the 
probability for the transition $\alpha_{1}\rightarrow\alpha_{2}$ (corresponding to  $\mathbf{C}\rightarrow\mathbf{C}^\sharp$), whereas $a_{1,8}$ expresses the probability for $\alpha_{1}\rightarrow\alpha_{8}$  (corresponding to  $\mathbf{C}\rightarrow\mathbf{G}$). In real music, the change from a tonic to the dominant is much more likely
than transposing by one semitone, so that the probability $a_{1,8}$ should be much larger than $a_{1,2}$. The coefficients $a_{i,i}$ express the probability of staying in state $\alpha_{i}$ (i.e., $\alpha_{i}\rightarrow\alpha_{i}$), $i\in[1:24]$. These coefficients are also referred to as **self-transition** probabilities.

A transition probability matrix can be specified in many ways. For example, the matrix may be defined manually by a music expert based on rules from harmony theory. The most common approach is to generate such a matrix automatically 
by estimating the transition probabilities from labeled data. 

In the following exercise, you will create a Markov model by determining transition probabilities found in the music of the Beatles using bigrams (pairs of adjacent elements) in labeled frame sequences from a subset of the Beatles Corpus.

For this assignment, assume each row in the dataset represents a chord that has followed the chord on the previous row in a Beatles song. When parsing the file for your model, you may discard any chord references beyond key and major/minor quality. For example, if a row reads 'Bbm7', you would parse for the key (B-flat) and quality (minor), but discard the extra information that it is a 7th chord.  

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from numpy.random import multinomial as randm
from numpy import where
import scipy.signal as si
import IPython.display as ipd
import librosa
import scipy
from matplotlib import patches
import librosa.display as ld
import music21
from music21 import midi as midi21
from music21 import stream
from jchord.progressions import ChordProgression, MidiConversionSettings
import copy
import matplotlib.pyplot as plt


np.random.seed(42)

data = pd.read_csv('Liverpool_band_chord_sequence.csv')

def preprocess(df):
    df[df['chords'] == 'Bb'] = 'A#'  
    chords = ' '.join(map(str, df['chords']))
    return chords

data = preprocess(data)


def play(x):
    """Returns nothing. Outputs a midi realization of x, a note or stream.
    Primarily for use in notebooks and web environments.
    """
    prog = ChordProgression.from_string(x)
    prog.to_midi(MidiConversionSettings(filename="example.midi", tempo=100, beats_per_chord=4, instrument=4))
    mf = midi21.MidiFile()
    mf.open("example.midi")
    mf.read()
    mf.close()
    s = midi21.translate.midiFileToStream(mf)
    myStream = stream.Stream()

    myStream.append(s)
    x = myStream
    
    if isinstance(x, stream.Stream):
        x = copy.deepcopy(x)
        for subStream in x.recurse(streamsOnly=True, includeSelf=True):
            mss = subStream.getElementsByClass(stream.Measure)
            for ms in mss:
                ms.offset += 1.0
    if isinstance(x, music21.note.Note):
        s = stream.Stream()
        s.append(music21.note.Rest(1))
        s.append(x)
        x = s
    x.show('midi')

##### Question 1 (20 points)

Using the above data, generate a 24x24 matrix, where each matrix element (i,j) is the transition probability from chord i to chord j. 

In [ ]:
MM = np.zeros((24,24))

### Your code here: 




In [ ]:
def plot_transition_matrix(A, ax=None, xlabel='State $a_j$', ylabel='State $a_i$', title='', clim=[-6, 0], cmap='gray_r'):
    
    im = plt.imshow(A, origin='lower', aspect='auto', cmap=cmap)
    im.set_clim([0, 1])
    cbar = plt.colorbar(im)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    cbar.ax.set_ylabel('Probability')
    
    chroma_label = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chord_label_maj = chroma_label
    chord_label_min = [s + 'm' for s in chroma_label]
    chord_labels = chord_label_maj + chord_label_min
    chord_labels_squeezed = chord_labels.copy()
    for k in [13, 15, 17, 18, 20, 22]:
        chord_labels_squeezed[k] = ''
        
    plt.xticks(np.arange(24), labels=chord_labels_squeezed )
    plt.yticks(np.arange(24), labels=chord_labels)
    
    return im

plot_transition_matrix(MM)

##### Question 2 (10 points)

Using your MM, you will create your own 16-measure Beatles hits!
For your first song, beginning on C major, select each next chord by choosing the chord with the largest transition probability from the current chord.

Make sure your chord progression string is formatted like this: 'C Dm G C'

Otherwise, it may not play in the in-browser MIDI player. 

In [ ]:
my_first_beatles_hit = ''

### Your code here:


print(my_first_beatles_hit)
play(my_first_beatles_hit)

##### Question 3 (10 points)

For your second song, beginning on C major, select each next chord at random according to the probabilities of your MM.
For example, if C major transitions to G major with probability .5, F major with probability .25, and D minor with probability .25, then your next chord should be selected randomly from (G, F, Dm) with probability of selection (.5, .25, .25) respectively. 

In [5]:
my_second_beatles_hit = ''

### Your code here:


print(my_second_beatles_hit)
play(my_second_beatles_hit)

## LZify: Applying Universal Prediction to Musical Style

LZify was the first algorithmic learning method to create a style immitation from a dictionary of motifs of variable size. It is based on the Lempel-Ziv compression method. Because the viriable context (motif) size that is used for perdiction of the next note, the method became also know as Variable Memory Markov model. Strictly speaking, is not a correct term, but LZ prediciton is known to perform asymptotically as good as any finite Markov model, so the terminology is partially justified. 

In this section, you will implement the Incremental Parsing (IP) from the Lempel Ziv LZ78 method for creating a dictionary of motifs. These motifs are later used to generate new sequences resembling the input sequence.

Please read the algorithm in Assayag, Dubnov, and Delerue's "Guessing the Composer’s Mind" (available at https://pdfs.semanticscholar.org/0181/236e1b417c8dd5dddd1f919583893f7a9026.pdf). 

The IPMotif function should compute the motif dictionary discovered in the text. It uses Incremental Parsing method to parse the text into unseen motifs.

##### Question 4 (20 points)

In [ ]:
def IPMotif(text):
    """Compute an associative dictionary (the motif dictionary)."""
    
    dictionary = {}
    
    ### Your Code Here:

    
    
    return dictionary


The text below contains an excerpt of Beethoven's Fur Elise, written as MML (Music Macro Language). MML is used to represent musical melodies as text. 

You can read more about MML syntax here: https://en.wikipedia.org/wiki/Music_Macro_Language

You can play with MML in this webapp: https://firecomb.github.io/final%20project.html

Try playing the text below in the webapp to hear sample output. 

In [ ]:
text = 'o6ed+ed+ec-dc>aceabeg+bb+e<ed+ed+ec-dc>aceabe<c>bab<cde>g<fed>f<edc>e<dc>be<eeed+ed+ec-dc>aceabeg+bb+e<ed+ed+ec-dc>aceabe<c>ba'
dict1 = IPMotif(text)
print(dict1)

Next, implement the IPContinuation and Normalize functions. 
The IPContinuation function transforms the IPMotif dictionary into a tree-like representation to allow finding continuations for new  motifs. The Normalize function turns the counters in every element of the IPContinuation dictionary into probabilities. 

##### Question 5 (20 points)

In [ ]:
def IPContinuation(dict1):
    """Compute continuation dictionary from a motif dictionary"""
    
    dict2 = {}

    ### Your Code Here: 
    
    
    return dict2

##### Question 6 (10 points)

In [ ]:
def Normalize(dict2):
    """Turns the counters in every element of dict2 to probabilities"""

    ### Your Code Here:
    

    return dict2

In [ ]:
dict2 = IPContinuation(dict1)
print(dict2)

Generting a new sequence is done by traversing the IPContinuation tree and selecting possible branches according to their weights. If motif is not found, its last symbol is removed and the process is repeated for a shorter motif.

In [ ]:
def IPGenerate(n,dict2):
    p = 0
    out = ""
    for k in range(n):
        while True:
            context = out[-p:]
            if context in dict2:
                prob = [tup[1] for tup in dict2[context]]
                conti = where(randm(1,prob))[0][0]
                cont = dict2[context][conti][0]
                out = out + cont
                break
            else:
                p = p-1
    return out
out = IPGenerate(92,dict2)
print(out)

##### Question 7 (10 points)

Paste your output in the online MML player, and listen to your piece. Do you hear elements of Fur Elise in your composition? What are some of the differences in the output from the original? 

Please export the WAV file of your output from the webapp, and submit the WAV file with your assignment. 

``` Your response here ```

A few important points:
1. The method captures the "texture" of the language but not it's meaning.
2. We could parse a new text using IPMotifs from two languages, then count the length and number of motifs in order to decide what was the language of the new text.
3. In order to use this method with musical information, we need first to translate audio to features, or in case of polyphonic midi change this into some proper representation. One possibility is using virtual fundamental or chroma for harmony, or some other specialized representation to capture repetition in terms of other specific musical properties.